# Development of Time Stepping
Tim Tyree<br>
7.16.2020

In [1]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  

#automate the boring stuff
from IPython import utils
import time, os, sys, re
beep = lambda x: os.system("echo -n '\\a';sleep 0.2;" * x)
if not 'nb_dir' in globals():
    nb_dir = os.getcwd()
sys.path.append("../lib") 
from lib import *
sys.path.append("lib") 
from lib import *

# from operari import *
# from ProgressBar import *
# from mesh_ops import *

# the visualization tools involved here for triangular meshes is
import trimesh
import pyglet
from numba import njit, cuda
# from numba.typed import List
# import numba
import trimesh

#try using a scipy sparse matrix to speed up spring force evaluations
#TODO: speed up bigger meshes with pycuda's sparce matrices
from scipy.sparse import csr_matrix
import scipy.sparse as sparse

from spring import *
from controller import *


# from pyspark import SparkContext 
# sc = SparkContext(master="local[4]")
# print(sc)


%autocall 1
%load_ext autoreload
%autoreload 2

Automatic calling is: Smart


## Scratch for Newmark time stepping

In [5]:
import numpy as np
from numpy import zeros,dot,atleast_1d
from scipy import linalg

In [7]:
# adapted from the following incumbent
##########################################################################
# program: Newmark.py
# author: Tom Irvine
# version: 1.4
# date: May 1, 2012
# description:  solution of a system of second-order ODEs for a dynamic
#               system via the Newmark-beta method
#
##########################################################################

In [8]:
def Newmark_coefficients(dt):
    alpha=0.25
    beta=0.5

    a0=1/(alpha*(dt**2))
    a1=beta/(alpha*dt)
    a2=1/(alpha*dt)
    a3=(1/(2*alpha))-1
    a4=(beta/alpha)-1
    a5=(dt/2)*((beta/alpha)-2)
    a6=dt*(1-beta)
    a7=beta*dt

    return a0,a1,a2,a3,a4,a5,a6,a7

In [15]:
Newmark_coefficients(dt=.05)

(1599.9999999999998, 40.0, 80.0, 1.0, 1.0, 0.0, 0.025, 0.025)

In [9]:
def Newmark_initialize(ndof,a0,a1,M,C,K,NT,DI,VI):

    nlength=len(atleast_1d(M)) 

    KH=zeros((ndof,ndof),float)

    KH=K+a0*M+a1*C

    if(nlength==1):

        U=zeros(NT,float)
        Ud=zeros(NT,float)
        Udd=zeros(NT,float)

        U[0]=DI
        Ud[0]=VI
            
    else:

        U=zeros((ndof,NT),float)
        Ud=zeros((ndof,NT),float)
        Udd=zeros((ndof,NT),float)

        U[:,0]=DI
        Ud[:,0]=VI

    return U,Ud,Udd,KH

In [10]:
def Newmark_force(M,C,K,FFI,force_dof,VI,DI,dt,NT,ndof):
    """
    input

    M = mass matrix
    C = damping matrix
    K = stiffness matrix

    FFI = interpolated force matrix
    force_dof = connects forces with dofs

    VI = initial velocity
    DI = initial displacement

      dt = time step
      NT = number of time points
    ndof = number of degrees of freedom

    output

      U = displacement
     Ud = velocity
    Udd = acceleration

    """

    a0,a1,a2,a3,a4,a5,a6,a7=Newmark_coefficients(dt)

    U,Ud,Udd,KH=Newmark_initialize(ndof,a0,a1,M,C,K,NT,DI,VI)


    for i in range (1,NT):

        V1=(a1*U[:,i-1]+a4*Ud[:,i-1]+a5*Udd[:,i-1])
        V2=(a0*U[:,i-1]+a2*Ud[:,i-1]+a3*Udd[:,i-1])

        CV=dot(C,V1)
        MA=dot(M,V2)


#  apply forces

        F=zeros(ndof,float)

        for j in range (0,ndof):

            j_index=force_dof[j]

            if(j_index!=-999):

                F[j]=FFI[i,j_index]


        FH=F+MA+CV

#  solve for displacements

        if(ndof>1):
            Un = linalg.solve(KH, FH)
        else:
            Un=FH/KH

        Uddn=a0*(Un-U[:,i-1])-a2*Ud[:,i-1]-a3*Udd[:,i-1]
        Udn=Ud[:,i-1]+a6*Udd[:,i-1]+a7*Uddn


        U[:,i]=Un
        Ud[:,i]=Udn
        Udd[:,i]=Uddn

    return U.T,Ud.T,Udd.T

In [17]:
def StarEnergy(self, others):
    retval = Newmark_initialize(ndof,a0,a1,M,C,K,NT,DI,VI) #TODO
    for other in others:
        retval += Change_in_Energy(self, others) #TODO
    return retval

# I need to stop being attention deficit

## TODO: thumb through the library of SublimeText and find an estimate of local energy for constitutive model, E(X) = _Energy_in_the_Star_radius_(X) is an extrinsic estimate while \Psi(X) is an intrinstic measure of the ground state that relaxes on a reletively slow electrophysiological time scale, 1ms.

$\tau_\text{EM}=\tau_\text{Electromotive}\sim1\,ms \ll 25\mu s\sim \tau_\text{EP}=\tau_\text{Electrophysiological}$

## Generate an array of randomized steps
TODO: try improving ^this model by using a different step size in the normal direction defined by the stable configuration given.

In [9]:
#TODO: define a numpy function that defines a random unbiased step for each vertex
#TODO: find the (i) the continuum constitutive relations and (i) the spring constitutive relations
#TODO: define an energy density function as the sum of those of the neighboring triangles
#TODO: define several energy functionals
#TODO: define an accept/reject energy functional

# mesh.vertices = time_step_n_times(
#     mesh.vertices,
#     X,
#     get_h(),
#     n=1)

X = mesh.vertices
#TODO: initialize displacement field to zero
U = mesh.vertices - X
def accept_step_query():
    pass

In [2]:
import trimesh, numpy as np, numba
from numba import njit,jit

In [8]:
#TODO: import mesh of sphere to define X
mesh_dir = nb_dir+"/Data/spherical_meshes/spherical_mesh_64.stl"
mesh = trimesh.load_mesh(mesh_dir)
X = mesh.vertices #\Omega_0 is the initially given configuration
x = X # set \Omega_{t=0} = \Omega_0
# mesh.show()

face_normals all zero, ignoring!


$$\Phi(X_i) \equiv \sum_{j=1}^{N_{X_i}}\Phi_j$$

Where $\Phi_j=$ is the energy density of the $j^{th}$ triangle neighboring the $i^{th}$ point $X_i$ located at $x(t)$ on the surface $\Omega_t$.

### define the energy of one triangle for the corotated linear model
from the notebook, "Elastic Model of RA", we have

In [ ]:
tet = np.stack((dm1,dm2,Am))
#half of the determinant of tet is the squared area of the triangle.
np.linalg.det(tet)/2==Am.dot(Am)
np.linalg.eigvals(tet)

## TODO: consider mapping between manifolds using scipy.optimize.minimize on some energy loss function.
methods str or callable, optional
Type of solver. Should be one of
- ‘Nelder-Mead’ (see here)
- ‘Powell’ (see here)
- ‘CG’ (see here)
- ‘BFGS’ (see here)
- ‘Newton-CG’ (see here)
- ‘L-BFGS-B’ (see here)
- ‘TNC’ (see here)
- ‘COBYLA’ (see here)
- ‘SLSQP’ (see here)
- ‘trust-constr’(see here)
- ‘dogleg’ (see here)
- ‘trust-ncg’ (see here)
- ‘trust-exact’ (see here)
- ‘trust-krylov’ (see here)
- custom - a callable object (added in version 0.14.0), see below for description.

In [13]:
from numba import vectorize
@vectorize('float32(float32, float32)')#, target='gpu')
def VectorAdd(a,b):
    return a + b
a = np.array(range(5),dtype=np.float32)


In [14]:
VectorAdd(a,a)

array([0., 2., 4., 6., 8.], dtype=float32)

## TODO: finish reading Augustin (2020).


See handwritten notes on Augustin (2020) 

which is a spacetime generalization of variational integrators that allows you to take different timesteps for different elements, which is important as slivers in the mesh often force you to take very small timesteps because of the CFL condition, and a global tensor product spacetime mesh would mean that the entire domain has to be advanced using that small timestep.

After today, I'll test that and see if I can reproduce the tests on an AVI on 2D flat space tessellated by triangles.  Once that works, I'll embed an example patient's atrium in 3D space, and tesselated it with triangles, and run the xgboost code on it, where I could smoothly track spiral tips on the atrial surface using a ray tracing projection onto the bezier surface generated by the patient's vertices as control knots.  


## TODO(after basic mvc pattern filled out): consider using a bezier surface to interpolate between the cardiac  surfaces that are updated at a time resolution of 1ms 

## TODO: write corotated linear strain energy functional to observe total strain energy of a given configuration
and then,

#TODO: create symplectic representation of the displacement and the rate of change of its displacement field, Q = (X, U)
#TODO: compute total energy for mesh H = (T+V)
#TODO: compute total action for mesh L = (T-V)

#TODO: minimize the action using scipy.optimize.minimize for any of the given methods
#TODO: if the first method doesn't work, try the next method.

In [44]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  

#automate the boring stuff
from IPython import utils
import time, os, sys, re
beep = lambda x: os.system("echo -n '\\a';sleep 0.2;" * x)
if not 'nb_dir' in globals():
    nb_dir = os.getcwd()
sys.path.append("../lib") 
from lib import *
sys.path.append("lib") 
from lib import *
from operari import *
from ProgressBar import *
# from mesh_ops import *

# the visualization tools involved here for triangular meshes is
import trimesh
import pyglet
from numba import njit, cuda
# from numba.typed import List
# import numba
import trimesh

#try using a scipy sparse matrix to speed up spring force evaluations
#TODO: speed up bigger meshes with pycuda's sparce matrices
from scipy.sparse import csr_matrix
import scipy.sparse as sparse

# from pyspark import SparkContext 
# sc = SparkContext(master="local[4]")
# print(sc)


%autocall 1
%load_ext autoreload
%autoreload 2

Automatic calling is: Smart
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [45]:
class BPSomeClass(object):
    r"""Describe the class"""
    def __init__(self, arg1, arg2):
        self.attr1 = arg1
        self.attr2 = arg2
    
    def attribute1(self):
        return self.attr1
bp_obj = BPSomeClass("a", 2.7182)
bp_obj.attribute1()

'a'

## TODO: use Newmark_force(M,C,K,FFI,force_dof,VI,DI,dt,NT,ndof) with no stiffness and fnet explicitely entered as the external force array.
- TODO: pipe compute_elastic_forces() into Newmark_force
   -  Hint: mdof_arbit_force_nm.py, control+F for Newmark_force
   
- is it worth taking advantage of the tree/queue data structures natively implemented in trimesh? mesh.nearest? 

In [ ]:
#TODO: copy opencarp into myfolder
#TODO: test out opencarp
#TODO: pycharm a 3D viewer app template
#TODO: bash your way into python
#use scipy.optimize.minimize to map one frame to another frame with the corotated linear constitutive model
#optimize lib.model.energy_functional \sim lib.model.action_functional using gekko


# Test driven dev of spring/edge manipulation.
# TODO: use ^this in removing edges closer than n edges of the given mesh boundary

In [233]:
def spring_exists(vid1_array, vid2_array, vid1,vid2):
    '''check where an edge exists'''
    return sum ( (vid1_array == vid1) & (vid2_array == vid2) ) > 0

def remove_springs(vid1_array, vid2_array, blacklist):
    '''
    remove a list of springs (blacklist) from the list of springs, (vid1_array,vid2_array) 
    found in Omegat.precomputed_arguments.
    
    vid1_array is an array of ints
    vid2_array is an array of ints
    blacklist is any input list of edges as a list of tuples of ints
    '''
    v1_list = []
    v2_list = []
    drop = (0.*vid1_array).astype(bool)
    for ax, bx in blacklist:
        boo   = (vid1_array == ax) & (vid2_array == bx) 
        drop |= boo
    return vid1_array[~drop],vid2_array[~drop]

def add_springs(vid1_array, vid2_array, whitelist):
    '''
    add a list of springs (blacklist) from the list of springs, (vid1_array,vid2_array) 
    found in Omegat.precomputed_arguments.
    
    vid1_array is an array of ints
    vid2_array is an array of ints
    blacklist is any input list of edges as a list of tuples of ints
    '''
    v1_list = list(vid1_array)
    v2_list = list(vid2_array)
    for ax, bx in whitelist:
        if not spring_exists(vid1_array, vid2_array, vid1=ax,vid2=bx):
            v1_list.append(ax)
            v2_list.append(bx)
    return np.array(v1_list, dtype=int), np.array(v2_list, dtype=int)



In [235]:
def test_spring_exists():
    # test spring_exists for a simple case
    assert ( spring_exists(np.array([3,4,1]), np.array([1,2,3]), vid1=3,vid2=1)     )
    assert ( not spring_exists(np.array([3,4,1]), np.array([1,2,3]), vid1=7,vid2=7) )
    assert ( not spring_exists(np.array([3,4,1]), np.array([1,2,3]), vid1=3,vid2=3) )
    assert ( not spring_exists(np.array([3,4,1]), np.array([1,2,3]), vid1=1,vid2=1) )
    return True
def test_spring_add_remove(Omegat):    
    #tests for adding/removing springs
    vid1_array = Omegat.precomputed_arguments['vid1_array']
    vid2_array = Omegat.precomputed_arguments['vid2_array']

    blacklist = [(7,7)]
    whitelist = [(7,7)]
    assert ( not spring_exists(vid1_array, vid2_array, vid1=7,vid2=7) ) 

    #test that add_springs and then remove_springs does nothing when told to do nothing
    len_original = 756
    assert ( len(vid1_array) == len_original )
    assert ( len(vid2_array) == len_original )

    v1, v2 = remove_springs(vid1_array, vid2_array, blacklist=[]) 
    assert ( len(v1) == len_original )
    assert ( len(v2) == len_original )

    v1, v2 = remove_springs(vid1_array, vid2_array, blacklist) 
    assert ( len(v1) == len_original )
    assert ( len(v2) == len_original )

    v1, v2 = add_springs(vid1_array, vid2_array, whitelist=[]) 
    assert ( len(v1) == len_original )
    assert ( len(v2) == len_original )

    v1, v2 = add_springs(v1, v2, whitelist) 
    assert ( len(v1) == len_original +1)
    assert ( len(v2) == len_original +1)
    assert ( spring_exists(v1, v2, vid1=7,vid2=7) )

    #add_springs does not add a spring that already exists
    v1, v2 = add_springs(v1, v2, whitelist) 
    assert ( len(v1) == len_original +1)
    assert ( len(v2) == len_original +1)
    assert ( spring_exists(v1, v2, vid1=7,vid2=7) )

    #remove_springs removes a spring that exists
    v1, v2 = remove_springs(v1, v2, whitelist) 
    assert ( len(v1) == len_original )
    assert ( len(v2) == len_original )
    assert ( not spring_exists(v1, v2, vid1=7,vid2=7) )
    return True


## TODO: implement time_step_n_steps() using forward euler integration.
- TODO: @njit time_step_n_steps()
- TODO: give example usage and other basic controller functions

In [15]:
# #Functional Goal: time step the mechanics n times 
# def time_step_n_times(x,X,h,n):
#     for k in range(n):
#         x += time_step(x,X,h)
#     return x
# def get_h(h = 10**-2):
#     '''constant time steps'''
#     return h
# # def get_h(h = 10**-2, beta = 1., acceptedQ=True):
# #     '''exponential adaptive time steps
# #     h = most recent time step,
# #     acceptedQ = whether the most recent time step was accepted,
# #     beta = stepsize change in parameter.
# #     **caution** this get_h could be unstable/inefficient with any unbiased R.W..
# #     Use it with steps directed towards the minimizer
# #     '''
# #     return h

# # #Example Usage: 
# # mesh.vertices = time_step_n_times(
# #     mesh.vertices,
# #     X,
# #     get_h(),
# #     n=1)

In [272]:
def precompute_mesh(mesh):
    '''precompute edges, which are the force generating simplices here'''
    Omega0 = mesh.copy()
    meu = mesh.edges_unique
    
    vid1_array, vid2_array = np.array(meu[:,0],dtype=int), np.array( meu[:,1],dtype=int)
    vertex1_array = np.array ( mesh.vertices[vid1_array]  , dtype=float) 
    vertex2_array = np.array ( mesh.vertices[vid2_array]  , dtype=float) 
    vertex_array =  np.array(Omega0.vertices, dtype=float) #only needed for shape
    
    
    #initialize spring force parameters
    d0_array = np.array(mesh.edges_unique_length, dtype=float)
    k0_array = 1 + np.zeros_like(vertex1_array[...,0], dtype=float)
    f0_array = np.array(np.zeros_like(mesh.vertices, dtype=float))

    #collect precomputed_arguments into a dict.
    precomputed_arguments = {
        'd0_array':d0_array, 
        'k0_array':k0_array, 
        'vid1_array':vid1_array, 
        'vid2_array':vid2_array, 
        'vertex_array':vertex_array,
        'mass_array':mass_array
     }

    vertex1_array = np.array( mesh.vertices[vid1_array],dtype=float)
    vertex2_array = np.array( mesh.vertices[vid2_array],dtype=float)
    d_array = np.array(mesh.edges_unique_length, dtype=float)
    recomputed_arguments = {
         'vertex1_array':vertex1_array, 
        'vertex2_array':vertex2_array, 
        'd_array':d_array, 
     }
    Omegat.precomputed_arguments = precomputed_arguments.copy()
    Omegat.recomputed_arguments  =  recomputed_arguments.copy()
    Omegat.X = vertex_array.copy()
    Omegat.Omega0 = Omega0.copy()
    return Omegat
    
def compute_mesh_update(Omegat,u):
    #compute the spring deformation states for each edge
        
    #update material vertices with material displacement field u
    mesh = Omegat.copy()
    mesh.vertices = mesh.Omega0.vertices + u 
    # ^this should not be mesh.X + u 
    #     (mesh.X removes tracked array from trimesh, which is made use of in mesh.edges_unique_length)
    
    #compute the spring deformation states for each edge
    vertex1_array = np.array( mesh.vertices[vid1_array],dtype=float)
    vertex2_array = np.array( mesh.vertices[vid2_array],dtype=float)
    d_array = np.array(mesh.edges_unique_length, dtype=float)
    
    recomputed_arguments = {
        'vertex1_array':vertex1_array, 
        'vertex2_array':vertex2_array, 
        'd_array':d_array, 
     }
    
    
    Omegat.precomputed_arguments = precomputed_arguments.copy()
    Omegat.recomputed_arguments  =  recomputed_arguments.copy()
    Omegat.X = vertex_array.copy()
    Omegat.Omega0 = Omega0.copy()
    
    return mesh

In [273]:
#TODO:functional goal: compute Vol, SVR timeseries
# TODO: get_dudt 
# TODO: get_fnet
# TODO: compute_masses of nodes
# TODO: initialize_
# TODO: briefly try find get_time_step outline?
# TODO: implement forward euler integration time_step()


In [17]:
def initialize_system(mesh):
    '''    initialize system fields
    X   = the material-space configuration
    u   = the displacement field
    ud  = the velocity field
    udd = the acceleration field'''

    #initialize configuration
    Omega0 = mesh.copy()
    Omegat = Omega0.copy()

    # intialize displacement field to zero
    X   = np.array(Omega0.vertices, dtype=float)
    xt  = np.array(Omegat.vertices, dtype=float)
    u   = np.zeros_like(X, dtype=float)
    
    # intialize velocity field to zero
    ud  = np.zeros_like(X, dtype=float)
    # intialize acceleration field to zero
    udd = np.zeros_like(X, dtype=float)

    #assert Omegat is still Omega0, which implies t = 0.
    assert ( np.isclose ( xt-X, np.zeros_like(X, dtype=float) ) .all() )  
    t = 0.                

    #initialize spring force parameters
    d0_array = np.array(mesh.edges_unique_length, dtype=float)
    k0_array = 1 + np.zeros_like(vertex1_array[...,0], dtype=float)
    #     f0_array = np.array(np.zeros_like(mesh.vertices, dtype=float))

    # collect precomputed_arguments into a dict.  
    # Warning: try to keep these arrays fixed throughout computation
    precomputed_arguments = {
        'd0_array':d0_array, 
        'k0_array':k0_array, 
        'vid1_array':vid1_array, 
        'vid2_array':vid2_array, 
        'vertex_array':X,
        'mass_array':mass_array
     }





    #compute the current spring deformation states for each edge
    # vertex1_array = Omegat[vid1_array].copy()


    
    Omega.precomputed_arguments = precomputed_arguments.copy()
    
    stdout = Omega0 
    
    return stdout

In [266]:
def recompute_system(Omegat, u, ud, udd ,Omega0, *recomputed_arguments):
    """this function is to be run after every n_steps from controller.py
    recomputed_arguments contains here
        'vid1_array':vid1_array, 
        'vid2_array':vid2_array, 
        'vertex_array':vertex_array,
        'mass_array':mass_array
    """
            
    #id's of the 1st and 2nd vertex of every edge (since edges generate forces here)
    vid1_array = Omegat.precomputed_arguments['vid1_array'].copy()
    vid2_array = Omegat.precomputed_arguments['vid2_array'].copy()
    
    
    try:
        Omegat.vold= Omegat.ud.copy()
    try:
        Omegat.aold= Omegat.udd.copy()
    try:
        Omegat.ud  = (Omegat.u-Omegat.vold)/dt
    try:
        Omegat.u   = Omegat.vertices - Omega0.vertices
        
    #TODO: compute fields in terms of Omegat.u = Omegat.vertices - Omega0.vertices
    vertex1_array = np.array(Omegat.vertices[vid1_array],dtype=float)
    vertex2_array = np.array( Omegat.vertices[vid2_array],dtype=float)
    d_array = np.array(Omegat.edges_unique_length, dtype=float)
    
    vertex1_array = np.array(Omegat.vertices[vid1_array],dtype=float)
    vertex2_array = np.array( Omegat.vertices[vid2_array],dtype=float)
    d_array = np.array(Omegat.edges_unique_length, dtype=float)
    distance_between = lambda  vertex1 , vertex2 : np.linalg.norm( vertex2 - vertex1 ) 
    d_array = np.array([distance_between ( vertex1 , vertex2 ) for vertex1,vertex2 in zip(vertex1_array, vertex2_array)], dtype=float)
    recomputed_arguments = {
        'vertex1_array':vertex1_array, 
        'vertex2_array':vertex2_array, 
        'd_array':d_array, 
     }
    

    
    Omegat.udd = (Omegat.ud-Omegat.aold)/dt
        
    Omega.recomputed_arguments = recomputed_arguments
    return 


SyntaxError: invalid syntax (<ipython-input-266-71c0962a5b23>, line 3)

In [3]:
def get_time_step(precomputed_arguments, h, mode=1, verbose=True):
    '''TODO: define the simplest productive time step 
    - unbiased R.W. that only accepts local improvements.
    - 1. FEI directed by elastic model.
    - 2. FEI directed by spring model.
    - 3. Newmark directed by elastic model.
    - 4. Newmark directed by spring model.
    - R.W. that only accepts local improvements, with steps directed spring/elastic model.
    TODO: define an array of njit'd time_step(h) functions for h in step_size_array
    - '''
    #         def get_forward_euler_integration_time_step(u, ud, X, anet_array, h, 
    #                                                     d0_array, k0_array, vid1_array, vid2_array, vertex_array):
    if mode==1:
        #precomputed model parameters don't need to be passed everytime to the time_step abstraction
        d0_array = precomputed_arguments['d0_array']
        k0_array = precomputed_arguments['k0_array']
        vid1_array  = precomputed_arguments['vid1_array']
        vid2_array  = precomputed_arguments['vid2_array']
        vertex_array  = precomputed_arguments['vertex_array']
        mass  = precomputed_arguments['mass_array']
        #arguments of the function returned
        retargs = ['h', 'vertex1_array', 'vertex2_array', 'd_array']
        if verbose:
            print ('explicit forward euler integration is used in the time_step function returned.')
            print (f"Example usage: u,ud = time_step(")
            for s in retargs:
                print (f"\t{s},")
            print (")")
        def get_anet_array_foo (vertex1_array, vertex2_array, d_array, d0_array, k0_array, vid1_array, vid2_array, vertex_array):
            '''compute the net spring force for each vertex.'''
            f0_array_foo   = lambda vertex1_array, vertex2_array, d_array: compute_spring_forces ( vertex1_array, vertex2_array, d_array, d0_array, k0_array, vid1_array, vid2_array, vertex_array)
#             fnet_array_foo = f0_array_foo  # + any other forces desired
#             anet_array_foo = np.divide ( fnet_array_foo , mass_array ) 
            #             fnet_array_foo = lambda vertex1_array, vertex2_array, d_array: f0_array_foo ( vertex1_array, vertex2_array, d_array) # + any other forces desired
            #             anet_array_foo = lambda vertex1_array, vertex2_array, d_array: np.divide ( fnet_array_foo ( vertex1_array, vertex2_array, d_array) , mass_array ) 
            return f0_array_foo 
            
        anet_array_foo = get_anet_array_foo (vertex1_array, vertex2_array, d_array, d0_array, k0_array, vid1_array, vid2_array, vertex_array)
        return anet_array_foo

#         @njit
        def get_d2udt2 ( anet_array_foo, vertex1_array, vertex2_array, d_array ):
            return anet_array_foo

#         @njit
        def get_dudt (h, anet_array_foo):
            '''returnds dudt.'''
            
            return lambda ud, vertex1_array, vertex2_array, d_array: ud + h * get_d2udt2 ( anet_array_foo, vertex1_array, vertex2_array, d_array )
#             dudt  = lambda ud, vertex1_array, vertex2_array, d_array: ud + h * udd ( vertex1_array, vertex2_array, d_array )
#             dudt  = ud + h * udd ( vertex1_array, vertex2_array, d_array )
#             return dudt

#         @njit
        def get_stepper (h, u, ud, anet_array_foo):
            '''example usage: 
            time_step = get_time_step (h, u, ud, anet_array_foo) 
            u_new = time_step(u, ud, vertex1_array, vertex2_array, d_array)
            TODO: write functional controller methods get_forward_integrate_n_steps
            TODO: minimalist test cases.  debug until it works
            TODO: njit this
            '''
            #                 dudt  = lambda ud, vertex1_array, vertex2_array, d_array: get_dudt (h, ud, anet_array_foo )
            dudt  = get_dudt (h, anet_array_foo )
#             time_step = lambda u, ud, vertex1_array, vertex2_array, d_array : u + np.multiply(h , get_dudt (h, ud, anet_array_foo ))
#             time_step = lambda u, ud, vertex1_array, vertex2_array, d_array : u + h * dudt (ud, vertex1_array, vertex2_array, d_array)
#             def time_step(u, ud, vertex1_array, vertex2_array, d_array ) : 
#                 return u + h * dudt (ud, vertex1_array, vertex2_array, d_array)
#             return dudt
            return dudt

#             return time_step, dudt
        return get_stepper (h, u, ud, anet_array_foo )   
#         time_step, dudt =  get_stepper (h, u, ud, anet_array_foo ) 
#         return time_step(u, ud, vertex1_array, vertex2_array, d_array)
    
    else:
        raise(f"Error! Method note implemented!")


In [228]:
#TODO: debug get_time_step until python can interpret it
# retval(ud, vertex1_array, vertex2_array, d_array)
#         anet_array_foo = get_anet_array_foo (vertex1_array, vertex2_array, d_array, d0_array, k0_array, vid1_array, vid2_array, vertex_array)

# foo = retval(ud, vertex1_array, vertex2_array, d_array)
# foo2= foo(ud, vertex1_array, vertex2_array, d_array)
# (ud, vertex1_array, vertex2_array, d_array)

In [22]:
#TODO: debug get_anet_array_foo until python can interpret it

def get_anet_array_foo (vertex1_array, vertex2_array, d_array, d0_array, k0_array, vid1_array, vid2_array, vertex_array):
    '''compute the net spring force for each vertex.'''
    f0_array_foo   = lambda vertex1_array, vertex2_array, d_array: compute_spring_forces ( vertex1_array, vertex2_array, d_array, d0_array, k0_array, vid1_array, vid2_array, vertex_array)
#             fnet_array_foo = f0_array_foo  # + any other forces desired
#             anet_array_foo = np.divide ( fnet_array_foo , mass_array ) 
    #             fnet_array_foo = lambda vertex1_array, vertex2_array, d_array: f0_array_foo ( vertex1_array, vertex2_array, d_array) # + any other forces desired
    #             anet_array_foo = lambda vertex1_array, vertex2_array, d_array: np.divide ( fnet_array_foo ( vertex1_array, vertex2_array, d_array) , mass_array ) 
    return f0_array_foo 


In [153]:
#compute the first time step
# TODO: update Omegat, time
# TODO: test ^this with a simple use case
# TODO: njit ^this!!!

In [132]:
# TODO: repeated forward time step n times
# TODO: test ^this with a simple use case
# TODO: njit ^this!!!

In [277]:
#Functional Goal: time step the mechanics n times 
def time_step_n_times(x,X,n,h):
    for k in range(n):
        x += time_step(x,X,h)
    return x
def get_h(h = 10**-2):
    '''constant time steps'''
    return h

# dev for simplest controller - FEI

TODO(controller-main): naive FEI is unstable.  try making the simulation stable with the following
- make the integration first order (viscous limit drag)
- keep second order integration, but include a drag
- use the IMR for position but not velocity, or use IMR in the first order version


- failing ^this, move on to optimizing the action/energy functional
    - shoudl the potential energy functional be spring or elastic?
    
    


TODO: test if njitQ {run time noticably?} controls whether or not njit is used on the n steps forward in time functionality
AFTER simplest controller has each part working, <br>
use FEI for each of a 
- get fnet_array
- get material position configuration fixed
- get displacement configuration and velocity configuration
- TODO: consider a 2D-interpolation to define the unit normal for pressure force calculations. for elastic force calculations?
return the updated displacement field and velocity field (Bonus: acceleration field)


- DONE: dev & test addition/removal of edges
- DONE: dev & test initialization of system

## dev for test at equilibrium

In [208]:
# #test getting use a constant time step size, h
# h = get_h(h=0.01)#, beta=1.0, acceptedQ=True)
# assert(h==0.01)

In [417]:
import os, trimesh, numpy as np
from lib.spring import *
os.chdir(f'Data/spherical_meshes')
mesh = trimesh.load('spherical_mesh_64.stl')
os.chdir('../..')

#subtract the center of mass
mesh.vertices -= mesh.center_mass
#normalize the mean radius to 1
mesh.vertices /= np.cbrt(mesh.volume*3/(4*np.pi))

#compute vertex masses for the displacement invariant barycentric discretization of a 2-D surface.
mass_array = np.array ( [get_mass(vid, mesh, density = 1.) for vid in range(mesh.vertices.shape[0])] )

Omegat = initialize_system(mesh, mass_array)
tme = 0.

face_normals all zero, ignoring!


In [418]:
# #test getting acceleration due to spring forces
# anet_foo =  get_anet_foo(Omegat.precomputed_arguments, h, mode=1, verbose=False)
# vertex1_array = Omegat.recomputed_arguments['vertex1_array']
# vertex2_array = Omegat.recomputed_arguments['vertex2_array']
# d_array       = Omegat.recomputed_arguments['d_array']
# assert ( anet_foo(vertex1_array, vertex2_array, d_array) is not None )
# assert ( not np.isnan( anet_foo(vertex1_array, vertex2_array, d_array) ).any() ) 


In [419]:
# #test all forces were correctly initialized to zero
# anet = anet_foo(vertex1_array, vertex2_array, d_array)
# assert ( (0 == anet).all() )
# assert ( (0 == anet_vertex1).all() )
# assert ( (0 == anet_vertex2).all() )

# #test that dipslacement, velocity, and acceleration fields are not nan
# assert ( not np.isnan(Omegat.u).any() ) 
# assert ( not np.isnan(Omegat.ud).any() ) 
# assert ( not np.isnan(Omegat.udd).any() ) 


In [258]:
# @njit
# def distance_3D(a, b):
#     '''a euclidian distance function in 3D real space'''
#     out  = (a[0]-b[0])**2
#     out += (a[1]-b[1])**2
#     out += (a[2]-b[2])**2
#     return np.sqrt(out)

# @njit
# def edges_unique_length(vertices1, vertices2):
#     '''returns the distances between vertices1 and vertices2'''
#     N = vertices1.shape[0]
#     d_array = np.zeros(N,dtype=np.float32)
#     for n in range(N):
#         d_array[n] = distance_3D(vertices1[n], vertices2[n])
#     return d_array 

In [421]:
# #test distance_3D
# a = np.array(range(3),dtype=np.float32)
# assert ( np.isclose(distance_3D(a, a+1) , np.sqrt(3) ) )

# # test edges_unique_length
# vertices1 = Omegat.recomputed_arguments['vertex1_array']
# vertices2 = Omegat.recomputed_arguments['vertex2_array']
# d_array   = edges_unique_length(vertices1, vertices2)

# assert ( d_array.shape == (756,) )

In [422]:
# def get_time_step(precomputed_arguments, h, mode=1, verbose=True, njitQ=True):
#     anet_foo = get_anet_foo(precomputed_arguments=precomputed_arguments, h=h, mode=mode, verbose=verbose)
#     vid1_array = precomputed_arguments['vid1_array']
#     vid2_array = precomputed_arguments['vid2_array']
#     X          = precomputed_arguments['vertex_array'] #X = material coordinates

#     def time_step(vertex1_array, vertex2_array, d_array, u, ud, udd, h):
#         #TODO(speed up time_step): move from edge basis to vertex basis only after all time steps occurred 
#         anet = anet_foo(vertex1_array, vertex2_array, d_array)
#         udd = anet
#         ud = ud + h*udd
#         u  = u + h*ud
#         x  = X + u
        
#         #update vertex1_array, vertex2_array, d_array
#         vertices1 = x[vid1_array]
#         vertices2 = x[vid2_array]
#         d_array = edges_unique_length(vertices1, vertices2)
#         return vertices1, vertices2, d_array, u, ud, udd, h
    
#     if njitQ:
#         return njit(time_step)
#     else:
#         return time_step

In [423]:
# #test that the time_step is not returning any nan values
# time_step = get_time_step(Omegat.precomputed_arguments, h=get_h(h=0.01), mode=1, verbose=False, njitQ=True)
# retval = time_step(vertex1_array, vertex2_array, d_array, u, ud, udd, h)
# for r in retval:
#     assert ( not np.isnan(r).any() ) 

In [425]:
def get_step_forward_n_times(precomputed_arguments, h, mode=1, verbose=True, njitQ=True):
    '''returns forward euler integration, get_step_backward_n_times(vertex1_array, vertex2_array, d_array, u, ud, udd, h, nsteps), which 
    returns its updated arguments, vertex1_array, vertex2_array, d_array, u, ud, udd, h'''
    time_step = get_time_step(precomputed_arguments=precomputed_arguments, h=h, mode=mode, verbose=verbose, njitQ=njitQ)
    
    def step_forward_n_times(vertex1_array, vertex2_array, d_array, u, ud, udd, h, nsteps):
        for n in range(int(nsteps)):
            vertex1_array, vertex2_array, d_array, u, ud, udd, h = time_step(vertex1_array, vertex2_array, d_array, u, ud, udd, h)
        return vertex1_array, vertex2_array, d_array, u, ud, udd, h
    
    if njitQ:
        return njit(step_forward_n_times)
    else:
        return step_forward_n_times

def get_step_backward_n_times(precomputed_arguments, h, mode=1, verbose=True, njitQ=True):
    '''returns naive backward euler integration, get_step_backward_n_times(vertex1_array, vertex2_array, d_array, u, ud, udd, h, nsteps), which 
    returns its updated arguments, vertex1_array, vertex2_array, d_array, u, ud, udd, h'''
    time_step = get_time_step(precomputed_arguments=precomputed_arguments, h=h, mode=mode, verbose=verbose, njitQ=njitQ)
    
    def step_backward_n_times(vertex1_array, vertex2_array, d_array, u, ud, udd, h, nsteps):
        for n in range(int(nsteps)):
            vertex1_array, vertex2_array, d_array, u, ud, udd, h = time_step(vertex1_array, vertex2_array, d_array, u, ud, udd, -h)
        return vertex1_array, vertex2_array, d_array, u, ud, udd, h
    
    if njitQ:
        return njit(step_backward_n_times)
    else:
        return step_backward_n_times

In [ ]:
#test stepping forward
nsteps = 100
h = get_h(h=0.01)
u  = Omegat.u
ud = Omegat.ud
udd= Omegat.udd
vertex1_array = Omegat.recomputed_arguments['vertex1_array']
vertex2_array = Omegat.recomputed_arguments['vertex2_array']
d_array   = edges_unique_length(vertex1_array, vertex2_array) 

step_forward_n_times = get_step_forward_n_times(Omegat.precomputed_arguments, h, mode=1, verbose=True, njitQ=True)

vertices1, vertices2, d_array, u, ud, udd, h = step_forward_n_times(vertex1_array, vertex2_array, d_array, u, ud, udd, h, nsteps)
tme += h*nsteps

#test that dipslacement, velocity, and acceleration fields are not nan
assert ( not np.isnan(u).any() ) 
assert ( not np.isnan(ud).any() ) 
assert ( not np.isnan(udd).any() ) 

Omegat.u   = u
Omegat.ud  = ud
Omegat.udd = udd
Omegat     = update_mesh(Omegat,u)

In [500]:
assert (Omegat)

## testing forward euler integration with nsteps (it's unstable)

In [501]:
import os, trimesh, numpy as np
from lib.spring import *
os.chdir(f'Data/spherical_meshes')
mesh = trimesh.load('spherical_mesh_64.stl')
os.chdir('../..')

#subtract the center of mass
mesh.vertices -= mesh.center_mass
#normalize the mean radius to 1
mesh.vertices /= np.cbrt(mesh.volume*3/(4*np.pi))

#compute vertex masses for the displacement invariant barycentric discretization of a 2-D surface.
mass_array = np.array ( [get_mass(vid, mesh, density = 1.) for vid in range(mesh.vertices.shape[0])] )

Omegat = initialize_system(mesh, mass_array)
tme = 0.

face_normals all zero, ignoring!


In [502]:
#specify spring force parameters for contracting all edges by 50%
vol_frac = 0.5
d0_array = vol_frac*np.array(mesh.edges_unique_length, dtype=float)
k0_array = np.ones_like(vertex1_array[...,0], dtype=float)

Omegat.precomputed_arguments['d0_array'] = d0_array.copy()
Omegat.precomputed_arguments['k0_array'] = k0_array.copy()

print(Omegat.volume)

4.18879020478639


In [503]:
nsteps = 100
h = get_h(h=0.001)
u  = Omegat.u
ud = Omegat.ud
udd= Omegat.udd
vertex1_array = Omegat.recomputed_arguments['vertex1_array']
vertex2_array = Omegat.recomputed_arguments['vertex2_array']
d_array   = edges_unique_length(vertex1_array, vertex2_array) 

step_forward_n_times = get_step_forward_n_times(Omegat.precomputed_arguments, h, mode=1, verbose=True, njitQ=True)

vertices1, vertices2, d_array, u, ud, udd, h = step_forward_n_times(vertex1_array, vertex2_array, d_array, u, ud, udd, h, nsteps)
tme += h*nsteps

#test that dipslacement, velocity, and acceleration fields are not nan
assert ( not np.isnan(u).any() ) 
assert ( not np.isnan(ud).any() ) 
assert ( not np.isnan(udd).any() ) 

Omegat.u   = u
Omegat.ud  = ud
Omegat.udd = udd
Omegat     = update_mesh(Omegat,u)

explicit forward euler integration is used in the time_step function returned.
Example usage: u,ud = time_step(
	h,
	vertex1_array,
	vertex2_array,
	d_array,
)


In [504]:
Omegat.volume

4.194445048568943

In [251]:
# #test that the shape of the displacement field is that of the vertices
# assert ( Omegat.u.shape == Omegat.vertices.shape ) 

In [361]:
# #test time step
# # time_step(vertices1, vertices2, d_array, u, ud, udd, h)
# ## def get_time_step(precomputed_arguments, h, mode=1, verbose=True):
# # # 	return get_anet_foo(precomputed_arguments=precomputed_arguments, h=mode, mode=mode, verbose=verbose)
# nsteps = 100000
# u  = Omegat.u
# ud = Omegat.ud
# udd= Omegat.udd

# def step_forward_n_times(vertex1_array, vertex2_array, d_array, u, ud, udd, h, n):
#     for n in range(nsteps):
#         vertices1, vertices2, d_array, u, ud, udd, h = time_step(vertex1_array, vertex2_array, d_array, u, ud, udd, h)
#     return vertices1, vertices2, d_array, u, ud, udd, h

# # vertices1, vertices2, d_array, u, ud, udd, h = step_forward_n_times(vertex1_array, vertex2_array, d_array, u, ud, udd, h,n)

# # Omegat.u   = u
# # Omegat.ud  = ud
# # Omegat.udd = udd
# # Omegat     = update_mesh(Omegat,u)

In [ ]:
#TODO: tests forward/backward stepping

#TODO: profile run n steps
#TODO: plot runtime on y axis and step number on x axis

#TODO: plot the volume of a contraction versus time



#TODO: test get_time_step


# get anet from a ball-and-spring model with barycentric mass weights and massless springs with spring constants k0_array and d0_array
#precompute/optimize pythonic spring-force calculations
# anet_array_foo = get_anet_array_foo (vertex1_array, vertex2_array, d_array, d0_array, k0_array, vid1_array, vid2_array, vertex_array)

#evaluate forces
# anet_array = anet_array_foo(vertex1_array, vertex2_array, d_array)
#notice that ^this can do 756 force computations in ~50 milliseconds, for sphere_64.stl, which has 756 edges.



In [84]:
#TODO: get_stepper
# #         @njit
# 		def get_stepper (h, u, ud, anet_array_foo):
# 			'''example usage: 
# 			time_step = get_time_step (h, u, ud, anet_array_foo) 
# 			u_new = time_step(u, ud, vertex1_array, vertex2_array, d_array)
# 			TODO: write functional controller methods get_forward_integrate_n_steps
# 			TODO: minimalist test cases.  debug until it works
# 			TODO: njit this
# 			'''
# 			#                 dudt  = lambda ud, vertex1_array, vertex2_array, d_array: get_dudt (h, ud, anet_array_foo )
# 			dudt  = get_dudt (h, anet_array_foo )
# #             time_step = lambda u, ud, vertex1_array, vertex2_array, d_array : u + np.multiply(h , get_dudt (h, ud, anet_array_foo ))
# #             time_step = lambda u, ud, vertex1_array, vertex2_array, d_array : u + h * dudt (ud, vertex1_array, vertex2_array, d_array)
# #             def time_step(u, ud, vertex1_array, vertex2_array, d_array ) : 
# #                 return u + h * dudt (ud, vertex1_array, vertex2_array, d_array)
# #             return dudt
# 			return dudt